<a href="https://colab.research.google.com/github/bachaudhry/FastAI-22-23/blob/main/FastAI_2022_Getting_Started_With_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Getting Started with NLP - Using FastAI and Hugging Face**


In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings, logging

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
warnings.simplefilter('ignore')
logging.disable(logging.WARNING)

In [2]:
# Setting up Kaggle so that we can download datasets directly
!pip install kaggle

In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 7.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [4]:
# Using Kaggle API key
creds = '{"username":"bachaudhry","key":"f8e0ee26f34cb0d3e4eccd774de62f63"}'

In [5]:
# Check if file path exists or needs to be created
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
  cred_path.parent.mkdir(exist_ok=True)
  cred_path.write_text(creds)
  cred_path.chmod(0o600)

Now that we have Kaggle setup for this NB, let's download the **US Patent Phrase Matching** dataset.

In [6]:
path = Path('us-patent-phrase-to-phrase-matching')

if not iskaggle and not path.exists():
  import zipfile, kaggle
  kaggle.api.competition_download_cli(str(path))
  zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 682k/682k [00:00<00:00, 99.9MB/s]

## **Import Data and EDA**

Here's a [description](https://https://www.kaggle.com/competitions/us-patent-phrase-to-phrase-matching/data) for the dataset that we'll be working on.

In short, we will be working on pairs of phrases - which consist of an `anchor` and a `target` phrase.

Additionally, similarity has been scored within a patent's `context`, which is a CPC classification and indicates the subject to which the patent relates.

In [7]:
# Checking local / GDrive path to verify files in the downloaded dataset.
!ls {path}

sample_submission.csv  test.csv  train.csv


In [8]:
# Loading training dataset in a DataFrame
df = pd.read_csv(path/'train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       36473 non-null  object 
 1   anchor   36473 non-null  object 
 2   target   36473 non-null  object 
 3   context  36473 non-null  object 
 4   score    36473 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.4+ MB


In [9]:
df.head(10)

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
5,067203128142739c,abatement,greenhouse gases,A47,0.25
6,061d17f04be2d1cf,abatement,increased rate,A47,0.25
7,e1f44e48399a2027,abatement,measurement level,A47,0.25
8,0a425937a3e86d10,abatement,minimising sounds,A47,0.50
9,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25


In [10]:
df.tail(10)

,id,anchor,target,context,score
36463,16a5c8551e534d1c,wood article,wood apple fruit,B44,0.00
36464,8ceaa2b5c2d56250,wood article,wood article,B44,1.00
36465,c4ac9d407fb427ab,wood article,wood logs,B44,0.50
36466,8a57100f6ee40ffc,wood article,wood material,B44,0.75
36467,f55e072f78d1fedb,wood article,wood substrate,B44,0.50
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75
36472,8d135da0b55b8c88,wood article,wooden substrate,B44,0.50


In [11]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186




We have 36473 rows, 733 unique anchors, 106 contexts and 29340 targets.





In [12]:
# Creating a column to concatenate the above so that we have a more convenient representation.
df['input'] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

## Tokenization Using HF Tokenizer

We will be working with HuggingFace's Tokenizer, which uses a `Dataset` object for storage.

In [13]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00


In [14]:
from datasets import Dataset, DatasetDict

In [15]:
# This is how a dataset object works when we load in the training DF.
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

It should be noted that Tokenization is dependent on the particular model that we will be using in this notebook. This detail has to be declared in the Tokenizer explicitly.

In [16]:
# Choosing a small NLP model for exploration
model_nm = 'microsoft/deberta-v3-small'

# Importing HuggingFace Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

In [17]:
tokz.tokenize("In the midst of winter, I found there was, within me, an invincible summer.")

['▁In',
 '▁the',
 '▁midst',
 '▁of',
 '▁winter',
 ',',
 '▁I',
 '▁found',
 '▁there',
 '▁was',
 ',',
 '▁within',
 '▁me',
 ',',
 '▁an',
 '▁invincible',
 '▁summer',
 '.']

In [18]:
tokz.tokenize("amazon.com, is a super-handy website. It's the bee's knees when it comes to e-commerce.")

['▁amazon',
 '.',
 'com',
 ',',
 '▁is',
 '▁a',
 '▁super',
 '-',
 'hand',
 'y',
 '▁website',
 '.',
 '▁It',
 "'",
 's',
 '▁the',
 '▁bee',
 "'",
 's',
 '▁knees',
 '▁when',
 '▁it',
 '▁comes',
 '▁to',
 '▁e',
 '-',
 'commerce',
 '.']

Let's tokenize our inputs column which we created in the previous section.

In [19]:
# Function to tokenize our inputs.
def tok_func(x):
  return tokz(x["input"])

In [20]:
# Testing the difference b/w batched=True and the default.
%time tok_ds = ds.map(tok_func)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

CPU times: user 13.6 s, sys: 150 ms, total: 13.8 s
Wall time: 22.2 s


In [21]:
%time tok_ds = ds.map(tok_func, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

CPU times: user 3.71 s, sys: 48 ms, total: 3.76 s
Wall time: 4.55 s


The performance gains with the usage of `batched` processing are quite pronounced...

In [22]:
round((4.5 - 22.2) / ((4.5 + 22.2) / 2) * 100, 2)

-132.58

... in this case, we have an improvement which is close to 132%.

Looking at the dataset again, we can see that there is a new column called `input_ids`.

In [23]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

These IDS come from a list called `vocab` in the tokenizer which contains unique integer for all token strings.

A random example of this can be:

In [24]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [25]:
# Checking vocab integers for random words.
tokz.vocab['▁needle'], tokz.vocab['▁the'], tokz.vocab['▁rain']

(9445, 262, 2894)

In [26]:
# NOTE: Transformers expects the labels column to be named labels by default.
# In our dataset this is refers to the `score` column.
tok_ds = tok_ds.rename_columns({'score': 'labels'})

In [27]:
# Checking all special tokens
tokz.all_special_tokens

['[CLS]', '[SEP]', '[UNK]', '[PAD]', '[MASK]']

## Test and Validation Sets

In [28]:
# Loading test set
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


The Transformers library uses a `Datasetdict` to hold our training and validation sets. It should be noted that here `validation` is labelled as `test`, so care must be taken.

In [29]:
# This was the first approach, which we will  swap for the manual approach below
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [30]:
# We will use eval as our name for the test set.
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
# Now, creating a dataset object based on the changes above.
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

### Creating Validation Sets Manually

In [44]:
anchors = df.anchor.unique()
np.random.seed(42)
np.random.shuffle(anchors)
anchors[:5]

array(['time digital signal', 'antiatherosclerotic', 'filled interior',
       'dispersed powder', 'locking formation'], dtype=object)

In [46]:
# Manually assigning a train and validation split
val_prop = 0.25
val_sz = int(len(anchors) * val_prop)
val_anchors = anchors[:val_sz]

In [47]:
is_val = np.isin(df.anchor, val_anchors)
idxs = np.arange(len(df))
trn_idxs = idxs[~is_val]
val_idxs = idxs[is_val]

len(trn_idxs), len(val_idxs)

(27357, 9116)

In [48]:
dds = DatasetDict({"train": tok_ds.select(trn_idxs),
                   "test" : tok_ds.select(val_idxs)})

In [49]:
# Comparing the mean scores for both train and validation sets
df.iloc[trn_idxs].score.mean(), df.iloc[val_idxs].score.mean()

(0.3623021530138539, 0.3613426941641071)

## Training an Initial Model

In [31]:
from transformers import TrainingArguments, Trainer

In [32]:
# Selecting a batch size, number of epochs and learning rate
bs = 128
epochs = 5
lr = 8e-5

In [50]:
# Setting up our training arguments using the class provided by HF
args = TrainingArguments('outputs',
                         learning_rate=lr,
                         warmup_ratio=0.1,
                         lr_scheduler_type='cosine',
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,
                         report_to='none')

In [51]:
# Function for our evaluation metric i.e. Pearson's Correlation
def corr(x, y):
  return np.corrcoef(x, y)[0][1]

def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}

In [52]:
# Defining the model object and the trainer.
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args,
                  train_dataset=dds['train'],
                  eval_dataset=dds['test'],
                  tokenizer=tokz,
                  compute_metrics=corr_d)

In [53]:
# Training the model
trainer.train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.032009,0.797658
2,No log,0.025751,0.807786
3,0.030000,0.025051,0.813243
4,0.030000,0.025478,0.813678
5,0.012200,0.025064,0.815026


TrainOutput(global_step=1070, training_loss=0.020381004620935315, metrics={'train_runtime': 259.9313, 'train_samples_per_second': 526.235, 'train_steps_per_second': 4.116, 'total_flos': 904509437110410.0, 'train_loss': 0.020381004620935315, 'epoch': 5.0})

Training for an increased number of epochs doesn't help the model at all.

In fact, the model appears to be overfitting to the training data since training loss decreases are occuring in conjunction with validation loss.

Based on the evaluation metric, Pearson's r doesn't improve beyond the 7th epoch.

Also, I experimented with both T4 and L4 GPUs annd the L4s essentially trained at 2-3x speed. The trade-off, however, is that we may end up burning through our allocated monthly compute!!!

## Improving the Model - A Primer

There aren't many improvements that can be made using the baseline modeling and data augmentation approach.

We can experiment with:

1. Improving the current model / selecting a larger model.
2. Different data augmentation approaches.
3. Experimenting with trainer parameters.

In [60]:
# Choosing our parameters
bs = 128
epochs = 5
lr = 8e-5
wd = 0.01
epochs = 5

In [54]:
inps = "anchor", "target", "context"

# Creating a function to quickly apply tokenization
def get_dds(df):
  ds = Dataset.from_pandas(df).rename_columns({'score': 'labels'})
  tok_ds = ds.map(tok_func,
                  batched=True,
                  remove_columns=inps + ('inputs', 'id'))
  return DatasetDict({"train": tok_ds.select(trn_idxs),
                      "test" : tok_ds.select(val_idxs)})

In [62]:
# Now we will create a function to create a Trainer.
def get_model():
  return AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)

def get_trainer(dds, model=None):
  if model is None:
    model = get_model()
    args = TrainingArguments('ouputs', learning_rate=lr,
                             warmup_ratio=0.1,
                             lr_scheduler_type='cosine',
                             fp16=True,
                             evaluation_strategy="epoch",
                             per_device_train_batch_size=bs,
                             per_device_eval_batch_size=bs*2,
                             num_train_epochs=epochs,
                             weight_decay=wd,
                             report_to='none')
  return Trainer(model, args, train_dataset=dds['train'],
                 eval_dataset=dds['test'], tokenizer=tokz,
                 compute_metrics=corr_d)



Jeremy thought that the presence of special characters wasn't a good idea and having more control over these auto generated elements might yield better results.

In [56]:
# The first iteration test basically swaps out the current separator for one
# which is defined by us.
sep = " [s] "
df['inputs'] = df.context + sep + df.anchor + sep + df.target
dds = get_dds(df)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [63]:
get_trainer(dds).train()

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.026578,0.798570
2,No log,0.025482,0.809249
3,0.031000,0.026066,0.813776
4,0.031000,0.025433,0.814491
5,0.012500,0.025002,0.815311


TrainOutput(global_step=1070, training_loss=0.021006357781240874, metrics={'train_runtime': 266.0299, 'train_samples_per_second': 514.171, 'train_steps_per_second': 4.022, 'total_flos': 904509437110410.0, 'train_loss': 0.021006357781240874, 'epoch': 5.0})

In [64]:
# Experimenting with lower case
df['inputs'] = df.inputs.str.lower()
dds = get_dds(df)
get_trainer(dds).train()

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.032009,0.797658
2,No log,0.025751,0.807786
3,0.030000,0.025051,0.813243
4,0.030000,0.025478,0.813678
5,0.012200,0.025064,0.815026


TrainOutput(global_step=1070, training_loss=0.020381004620935315, metrics={'train_runtime': 269.3715, 'train_samples_per_second': 507.793, 'train_steps_per_second': 3.972, 'total_flos': 904509437110410.0, 'train_loss': 0.020381004620935315, 'epoch': 5.0})

Jeremy also experimented with treating the patent section as a special token.

This may allow the model to learn to handle various sections of the dataset differently.

In [66]:
df['section'] = df.context.str[0]
df.section.value_counts()

section
B    8019
H    6195
G    6013
C    5288
A    4094
F    4054
E    1531
D    1279
Name: count, dtype: int64

In [67]:
df['sectok'] = '[' + df.section + ']'
sectoks = list(df.sectok.unique())
tokz.add_special_tokens({'additional_special_tokens':  sectoks})

8

In [69]:
df['inputs'] = df.sectok + sep + df.context + sep + df.anchor.str.lower() + sep + df.target
dds = get_dds(df)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [70]:
# We will need to resize the embeddings matrix in the model since new tokens are being added
model = get_model()
model.resize_token_embeddings(len(tokz))

Embedding(128009, 768)